In [9]:
# -*- coding: utf-8 -*-
# @Time    : 2019/05
# @Author  : zhangle/tongxusheng
# @function   : data mining
# @Software: Jupyter

import pandas as pd
import multiprocessing
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

data= pd.read_csv('data.csv')
train = data.iloc[:543990,:]
test = data.iloc[-200000:,:]
print(train.shape)
print(test.shape)

C:\Software\anaconda\ana\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4,5,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(543990, 27)
(200000, 27)


In [10]:
#数据预处理
train = train[train.gender != '\\N']
test = test[test.gender != '\\N']
train['gender'] = train['gender'].apply(lambda x : int(x))
test['gender'] = test['gender'].apply(lambda x : int(x))

train = train[train.age != '\\N']
test = test[test.age != '\\N']
train['age'] = train['age'].apply(lambda x : int(x))
test['age'] = test['age'].apply(lambda x : int(x))

train = train[train['2_total_fee'] != '\\N']
test = test[test['2_total_fee'] != '\\N']
test.loc[test['2_total_fee'] == '\\N','2_total_fee'] = 0.0
train['2_total_fee'] = train['2_total_fee'].apply(lambda x : float(x))
test['2_total_fee'] = test['2_total_fee'].apply(lambda x : float(x))

train = train[train['3_total_fee'] != '\\N']
test = test[test['3_total_fee'] != '\\N']
test.loc[test['3_total_fee'] == '\\N','3_total_fee'] = 0.0
train['3_total_fee'] = train['3_total_fee'].apply(lambda x : float(x))
test['3_total_fee'] = test['3_total_fee'].apply(lambda x : float(x))
data = pd.concat([train, test]).reset_index(drop=True).fillna(0) #合并之后重新建立索引 将空缺的地方补为0
data.head()

,service_type,is_mix_service,online_time,1_total_fee,2_total_fee,3_total_fee,4_total_fee,month_traffic,many_over_bill,contract_type,...,local_caller_time,service1_caller_time,service2_caller_time,gender,age,complaint_level,former_complaint_num,former_complaint_fee,current_service,user_id
0,4,0,85,295.96,296.20,296.00,296.80,3813.614698,0,1,...,108.100000,0.000000,564.366667,1,31,0,0,0.0,99999825,Gx4sJzcQog01UhZL
1,1,0,10,265.20,261.20,208.50,174.50,0.000000,1,0,...,240.100000,355.166667,0.000000,1,30,0,0,0.0,90063345,kEXrhTiug93DIcLG
2,1,0,12,44.50,70.20,69.00,61.40,2598.397406,0,0,...,27.666667,0.000000,98.416667,2,25,0,0,0.0,90109916,AouXr0EOUtSRdiYK
3,4,0,134,87.95,81.40,76.00,88.30,988.440563,0,0,...,89.900000,74.483333,121.833333,1,44,0,0,0.0,89950166,Yds7U30hnRZDiLtb
4,4,0,84,317.04,314.08,435.51,413.05,5885.800642,0,1,...,0.000000,35.100000,496.733333,1,31,2,1,0.0,99999825,OFDTSXrhN9Q2mbVw


In [11]:
current_service = train.current_service.values
set(current_service)
#一共有11种类型

{89950166,
 89950167,
 89950168,
 90063345,
 90109916,
 90155946,
 99999825,
 99999826,
 99999827,
 99999828,
 99999830}

In [12]:
def LGB():
        clf = lgb.LGBMClassifier(
                bjective='multiclass',
                boosting_type='gbdt',
                num_leaves=35,
                max_depth=8,
                learning_rate=0.05,
                seed=2018,
                colsample_bytree=0.8,
                subsample=0.9,
                n_estimators=1000)
        return clf

In [13]:
#准备 全模型 训练集
Xtrain = train.drop(['user_id'],axis = 1)
Xtrain_feature = [value for value in Xtrain.columns.values if
                   value not in ['current_service']]
Xtrain[Xtrain_feature].head()
Xtrain_feature

['service_type',
 'is_mix_service',
 'online_time',
 '1_total_fee',
 '2_total_fee',
 '3_total_fee',
 '4_total_fee',
 'month_traffic',
 'many_over_bill',
 'contract_type',
 'contract_time',
 'is_promise_low_consume',
 'net_service',
 'pay_times',
 'pay_num',
 'last_month_traffic',
 'local_trafffic_month',
 'local_caller_time',
 'service1_caller_time',
 'service2_caller_time',
 'gender',
 'age',
 'complaint_level',
 'former_complaint_num',
 'former_complaint_fee']

In [14]:
#全模型标签
label = Xtrain['current_service']
le = LabelEncoder()
label = le.fit_transform(label)
print(set(label))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}


In [15]:
#全模型测试集
Xtest = test.drop(['user_id'],axis = 1)
Xtest_feature = [value for value in Xtest.columns.values if
                   value not in ['current_service']]
Xtest[Xtest_feature].head()

,service_type,is_mix_service,online_time,1_total_fee,2_total_fee,3_total_fee,4_total_fee,month_traffic,many_over_bill,contract_type,...,last_month_traffic,local_trafffic_month,local_caller_time,service1_caller_time,service2_caller_time,gender,age,complaint_level,former_complaint_num,former_complaint_fee
543990,1,0,15,49.1,50.9,44.50,57.60,0.000000,1,0,...,0.000000,0.00000,0.00,0.0,6.666667,1,30,0,0,0.0
543991,4,0,14,62.0,183.3,43.60,67.76,548.944325,0,0,...,0.000000,0.00000,0.00,0.0,51.466667,2,17,0,0,0.0
543992,1,0,6,36.0,36.0,36.00,43.70,2135.174574,0,0,...,943.416368,2216.05873,2.25,0.0,87.233333,1,22,0,0,0.0
543993,1,0,7,78.5,58.0,45.50,51.10,0.000000,1,0,...,0.000000,0.00000,0.00,78.8,4.766667,2,23,0,0,0.0
543994,1,0,4,60.4,64.9,33.29,0.00,0.000000,1,0,...,0.000000,21047.09120,86.80,86.8,0.000000,2,17,0,0,0.0


In [16]:
#全模型训练-决策树
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.tree import export_graphviz
import pydot
#使用信息熵作为划分标准，对决策树进行训练
#确定参数
'''
pipeline = tree.DecisionTreeClassifier(criterion='entropy')
parameters = {
    'max_depth':(8,9,10),    
    'min_samples_split':(2,3,4),
    'min_samples_leaf':(1,2,3,4)
}
grid_search = GridSearchCV(pipeline,parameters,cv=5) #网格搜索+交叉验证
grid_search.fit(Xtrain[Xtrain_feature], label)
best_parameters = grid_search.best_params_
print('最佳参数:',best_parameters)
DT = tree.DecisionTreeClassifier(**best_parameters) 
'''
DT = tree.DecisionTreeClassifier(criterion='entropy')  
DT.fit(Xtrain[Xtrain_feature], label)  
print(DT)  
#测试
y_DT = DT.predict(Xtest[Xtest_feature])
pred_DT = le.inverse_transform(y_DT)
test['predict_DT'] = pred_DT

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')


C:\Software\anaconda\ana\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [17]:
#全模型训练-随机森林
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier()    
RF.fit(Xtrain[Xtrain_feature], label)  
print(RF)
#测试
y_RF = RF.predict(Xtest[Xtest_feature])
pred_RF = le.inverse_transform(y_RF)
test['predict_RF'] = pred_RF

C:\Software\anaconda\ana\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


C:\Software\anaconda\ana\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [18]:
#全模型训练
lgb = LGB()
lgb.fit(Xtrain[Xtrain_feature], label, eval_set=[(Xtrain[Xtrain_feature], label)], verbose=1)
pred_lgb = lgb.predict(Xtest[Xtest_feature])
pred_lgb = le.inverse_transform(pred_lgb)
test['predict_lgb'] = pred_lgb

[1]	valid_0's multi_logloss: 1.77735
[2]	valid_0's multi_logloss: 1.64953
[3]	valid_0's multi_logloss: 1.55038
[4]	valid_0's multi_logloss: 1.46407
[5]	valid_0's multi_logloss: 1.39033
[6]	valid_0's multi_logloss: 1.32372
[7]	valid_0's multi_logloss: 1.26097
[8]	valid_0's multi_logloss: 1.20468
[9]	valid_0's multi_logloss: 1.15345
[10]	valid_0's multi_logloss: 1.10711
[11]	valid_0's multi_logloss: 1.06298
[12]	valid_0's multi_logloss: 1.02238
[13]	valid_0's multi_logloss: 0.984922
[14]	valid_0's multi_logloss: 0.949371
[15]	valid_0's multi_logloss: 0.916484
[16]	valid_0's multi_logloss: 0.885115
[17]	valid_0's multi_logloss: 0.856529
[18]	valid_0's multi_logloss: 0.828406
[19]	valid_0's multi_logloss: 0.802809
[20]	valid_0's multi_logloss: 0.778124
[21]	valid_0's multi_logloss: 0.75514
[22]	valid_0's multi_logloss: 0.733006
[23]	valid_0's multi_logloss: 0.712075
[24]	valid_0's multi_logloss: 0.693017
[25]	valid_0's multi_logloss: 0.674525
[26]	valid_0's multi_logloss: 0.65636
[27]	vali

[210]	valid_0's multi_logloss: 0.204017
[211]	valid_0's multi_logloss: 0.203766
[212]	valid_0's multi_logloss: 0.203491
[213]	valid_0's multi_logloss: 0.203248
[214]	valid_0's multi_logloss: 0.202989
[215]	valid_0's multi_logloss: 0.202709
[216]	valid_0's multi_logloss: 0.202454
[217]	valid_0's multi_logloss: 0.202216
[218]	valid_0's multi_logloss: 0.20194
[219]	valid_0's multi_logloss: 0.201693
[220]	valid_0's multi_logloss: 0.201436
[221]	valid_0's multi_logloss: 0.20121
[222]	valid_0's multi_logloss: 0.200979
[223]	valid_0's multi_logloss: 0.200743
[224]	valid_0's multi_logloss: 0.200522
[225]	valid_0's multi_logloss: 0.20026
[226]	valid_0's multi_logloss: 0.200033
[227]	valid_0's multi_logloss: 0.199826
[228]	valid_0's multi_logloss: 0.199585
[229]	valid_0's multi_logloss: 0.199351
[230]	valid_0's multi_logloss: 0.19911
[231]	valid_0's multi_logloss: 0.198868
[232]	valid_0's multi_logloss: 0.198647
[233]	valid_0's multi_logloss: 0.198407
[234]	valid_0's multi_logloss: 0.198178
[235

[416]	valid_0's multi_logloss: 0.170552
[417]	valid_0's multi_logloss: 0.170454
[418]	valid_0's multi_logloss: 0.170359
[419]	valid_0's multi_logloss: 0.170248
[420]	valid_0's multi_logloss: 0.170129
[421]	valid_0's multi_logloss: 0.170023
[422]	valid_0's multi_logloss: 0.169914
[423]	valid_0's multi_logloss: 0.169816
[424]	valid_0's multi_logloss: 0.169711
[425]	valid_0's multi_logloss: 0.169608
[426]	valid_0's multi_logloss: 0.169511
[427]	valid_0's multi_logloss: 0.169401
[428]	valid_0's multi_logloss: 0.169297
[429]	valid_0's multi_logloss: 0.169188
[430]	valid_0's multi_logloss: 0.169087
[431]	valid_0's multi_logloss: 0.168991
[432]	valid_0's multi_logloss: 0.168887
[433]	valid_0's multi_logloss: 0.168779
[434]	valid_0's multi_logloss: 0.16866
[435]	valid_0's multi_logloss: 0.168566
[436]	valid_0's multi_logloss: 0.16847
[437]	valid_0's multi_logloss: 0.168346
[438]	valid_0's multi_logloss: 0.168244
[439]	valid_0's multi_logloss: 0.168131
[440]	valid_0's multi_logloss: 0.168013
[4

[622]	valid_0's multi_logloss: 0.152319
[623]	valid_0's multi_logloss: 0.152251
[624]	valid_0's multi_logloss: 0.152182
[625]	valid_0's multi_logloss: 0.152105
[626]	valid_0's multi_logloss: 0.152033
[627]	valid_0's multi_logloss: 0.151951
[628]	valid_0's multi_logloss: 0.151878
[629]	valid_0's multi_logloss: 0.151818
[630]	valid_0's multi_logloss: 0.151737
[631]	valid_0's multi_logloss: 0.151662
[632]	valid_0's multi_logloss: 0.151593
[633]	valid_0's multi_logloss: 0.15152
[634]	valid_0's multi_logloss: 0.151454
[635]	valid_0's multi_logloss: 0.151387
[636]	valid_0's multi_logloss: 0.151316
[637]	valid_0's multi_logloss: 0.151251
[638]	valid_0's multi_logloss: 0.151178
[639]	valid_0's multi_logloss: 0.151114
[640]	valid_0's multi_logloss: 0.15105
[641]	valid_0's multi_logloss: 0.150984
[642]	valid_0's multi_logloss: 0.150905
[643]	valid_0's multi_logloss: 0.15084
[644]	valid_0's multi_logloss: 0.150775
[645]	valid_0's multi_logloss: 0.150706
[646]	valid_0's multi_logloss: 0.15064
[647

[828]	valid_0's multi_logloss: 0.138946
[829]	valid_0's multi_logloss: 0.138886
[830]	valid_0's multi_logloss: 0.138832
[831]	valid_0's multi_logloss: 0.13877
[832]	valid_0's multi_logloss: 0.13871
[833]	valid_0's multi_logloss: 0.138651
[834]	valid_0's multi_logloss: 0.138588
[835]	valid_0's multi_logloss: 0.138529
[836]	valid_0's multi_logloss: 0.138478
[837]	valid_0's multi_logloss: 0.138417
[838]	valid_0's multi_logloss: 0.138358
[839]	valid_0's multi_logloss: 0.138296
[840]	valid_0's multi_logloss: 0.138237
[841]	valid_0's multi_logloss: 0.13818
[842]	valid_0's multi_logloss: 0.138117
[843]	valid_0's multi_logloss: 0.138056
[844]	valid_0's multi_logloss: 0.138003
[845]	valid_0's multi_logloss: 0.137944
[846]	valid_0's multi_logloss: 0.137891
[847]	valid_0's multi_logloss: 0.137834
[848]	valid_0's multi_logloss: 0.137786
[849]	valid_0's multi_logloss: 0.137718
[850]	valid_0's multi_logloss: 0.137659
[851]	valid_0's multi_logloss: 0.13759
[852]	valid_0's multi_logloss: 0.137519
[853

C:\Software\anaconda\ana\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Software\anaconda\ana\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [19]:
#F1与准确率
y_true = test['current_service'].tolist()
y1 = test['predict_DT'].tolist()
y2 = test['predict_RF'].tolist()
y3 = test['predict_lgb'].tolist()
print('Decision Tree F1 score: ',f1_score(y_true, y1, average='macro'))
print('Random Forest F1 score: ',f1_score(y_true, y2, average='macro'))
print('lightgbm F1 score: ',f1_score(y_true, y3, average='macro'))
y = [y1,y2,y3]
for y_pred in y:
    acc = 0
    for i in range(len(y_pred)):
        if(y_pred[i] ==y_true[i]):
            acc+=1
    print('Accuracy: ',acc/len(y_pred))

Decision Tree F1 score:  0.8565266127502592
Random Forest F1 score:  0.8807751917815817
lightgbm F1 score:  0.9006891717277032
Accuracy:  0.9074540745407454
Accuracy:  0.9244342443424435
Accuracy:  0.9363743637436375
